# 1. JOBS NO DEUCALION (A PARTIR DO JUPYTER)

> Objetivo: preparar, comentar e submeter um job simples de MPI em Python, validando conta, módulos, e recursos ARM, a partir de células no Jupyter.

## 1.1 Verificar conta e associações SLURM

In [ ]:
!whoami                          # mostra o nome do username
!billing                         # mostra a(s) conta(s) de billing com o nome completo

## 1.2 Confirmar ambiente e módulos atuais

In [ ]:
!hostname      # Verifica em que nó se está ligado (ex.: ln01). Útil para confirmar que é o login node.
!which python  # Caminho absoluto do executável Python ativo. Deve apontar para o Python do módulo carregado
!python -V     # Versão exata do Python atualmente em uso. Confirma se corresponde ao módulo carregado.
!module list   # Lista a pilha de módulos carregados na sessão do kernel (herdados). 

## 1.3 Ver recursos: login node vs nós ARM

In [ ]:
!lscpu

Este comando refere-se ao *login node*.

Em Python MPI lidamos com um processo por CPU Core. Não usamos Threads.

Observando o resultado:

 - CPU(s):              256
 - Thread(s) per core:  2
 - Core(s) per socket:  64
 - Socket(s):           2
 
>Como existem **dois Threads por CPU Core** vemos 256 CPU's lógicos.

>Na realidade temos 2 sockets com **64 CPU Cores** cada um, logo temos disponíveis **128 CPU Cores**.

>Isto permite-nos ter **128 processos a correr em paralelo** pois fica cada um a correr num CPU Core.


In [ ]:
!sacctmgr -Pn show assoc where user=$USER format=Account,Partition #Verifica quais as nossas contas e se tem restrições de nodes

In [ ]:
!sacctmgr -Pn show assoc where user=$USER format=Account,Partition,QOS # Acrescenta o sistema de prioridades

In [ ]:
!sinfo # Mostra a lista de partições e nós, estado, etc..

**Interpretação da saída do comando sinfo**

>O comando **sinfo** apresenta o estado das partições do cluster, agregando informação operacional relevante para planear a submissão de jobs.

Campos principais

>PARTITION — designação da partição (ex.: dev-arm, normal-arm, large-arm, *-x86, *-a100-*).

>AVAIL — disponibilidade atual da partição (ex.: up).

>TIMELIMIT — tempo máximo permitido por job na partição (ex.: 4:00:00, 2-00:00:00).

>NODES — número de nós incluídos na linha (com o estado indicado).

>STATE — estado operacional dos nós agregados nessa linha:

>idle (livres), alloc (totalmente alocados), mix (parcialmente alocados),

>down (indisponíveis), drain/drain* (retirados da fila), comp (em conclusão/remoção).

>NODELIST — identificação dos nós, possivelmente compactada por intervalos (ex.: cna[0052-1547]).

**Leitura e decisões**

>Arquitetura: partições *-arm correspondem a nós CPU ARM; *-x86 a nós CPU x86; *-a100-* a nós GPU (NVIDIA A100).

>Capacidade e carga: a presença de linhas distintas por partição com estados diferentes (p. ex., alloc e idle) indica, respetivamente, nós ocupados e nós disponíveis.
Exemplo: normal-arm … 111 alloc e normal-arm … 1501 idle significam 111 nós ocupados e 1501 nós disponíveis nessa partição.

>Limites temporais: TIMELIMIT define o teto de execução por job nessa partição e deve ser compatível com a duração prevista do trabalho.

>Elegibilidade: o acesso efetivo a uma partição pode ainda ser condicionado por políticas (AllowAccounts, AllowQos, etc.). A ausência de restrição na associação do utilizador/conta não garante, por si só, elegibilidade se a partição impuser filtros.

**Estados (glossário sucinto)**

| STATE            | Interpretação técnica                  |
| ---------------- | -------------------------------------- |
| `idle`           | Nó disponível para alocação de jobs    |
| `alloc`          | Nó integralmente alocado               |
| `mix`            | Nó parcialmente alocado                |
| `down`           | Nó indisponível                        |
| `drain`/`drain*` | Nó retirado para manutenção/isolamento |
| `comp`           | Nó em fase de conclusão/remoção        |



## 1.4 Criar o script Python (MPI “Hello World”, 4 processos)

Célula Jupyter para criar o ficheiro hello_mpi.py.

In [ ]:
%%writefile hello_mpi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
host = MPI.Get_processor_name()

print(f"Hello world from process {rank} of {size} on {host}")

**O que fazem as linhas do programa acima? - base do MPI (Message Passing Interface)**

>%%writefile hello_mpi.py — diretiva do Jupyter que grava todo o conteúdo da célula num ficheiro chamado hello_mpi.py. Serve apenas para criar o script em disco a partir do notebook.

>from mpi4py import MPI — importa o módulo MPI da biblioteca mpi4py, que expõe a interface padrão do MPI (communicators, ranks, envios/receções, operações coletivas) para uso em Python.

>comm = MPI.COMM_WORLD — obtém o communicator global que inclui todos os processos do programa (todos os ranks do job). É o “canal” comum de comunicação entre processos.

>rank = comm.Get_rank() — devolve o identificador inteiro deste processo dentro do communicator (rank), de 0 a size−1. Permite distinguir comportamentos por processo (por exemplo, apenas o rank 0 fazer I/O agregado).

>size = comm.Get_size() — devolve o número total de processos presentes no communicator (tamanho do grupo). Útil para dimensionar decomposições de dados e controlos de fluxo.

>host = MPI.Get_processor_name() — devolve o nome do nó físico onde este processo está a correr (por exemplo, cna0004…). Ajuda a confirmar a distribuição pelos nós do cluster.

>print(f"Hello world from process {rank} of {size} on {host}") — imprime uma linha por processo com o rank, o total de processos e o nome do nó, confirmando que o programa está a executar em paralelo e de forma distribuída. Observação: a ordem das linhas no output pode não ser sequencial (0,1,2,3,…) porque todos os processos escrevem em paralelo para stdout.

## 1.5 Criar o job script SLURM

Célula Jupyter para criar o ficheiro job_hello.sh.
Nota: o carregamento de módulos faz-se dentro do job para garantir o stack correto nos nós ARM (o login é x86).

In [ ]:
%%writefile job_hello.sh
#!/bin/bash
#SBATCH --job-name=hello4                 # Identificação do job
#SBATCH --account=f202500003hpcvlabutada  # Conta/billing a usar
#SBATCH --partition=dev-arm               # Partição ARM (trocar para normal-arm se necessário)
#SBATCH --ntasks=4                        # Nº total de processos MPI (ranks)
#SBATCH --cpus-per-task=1                 # 1 core por rank (MPI puro)
#SBATCH --time=00:02:00                   # Limite temporal (curto, demonstração)
#SBATCH --output=hello.txt                # Captura stdout (um ficheiro por job)

# Garantir ambiente ARM coerente no compute node (não herdar do login x86)
module purge
module load foss/2022a
module load Python/3.10.4-GCCcore-11.3.0
module load SciPy-bundle/2022.05-foss-2022a
# module load mpi4py ---> contido em SciPy-bundle

# Lançamento recomendado em SLURM: 'srun' integra alocação, binding e PMIx/PMI
# #SBATCH --ntasks=4 reserva 4 tarefas (ranks) para o job.
#srun lança um job step dentro desse job.
#Por omissão, srun usa 1 tarefa se não lhe "disserem" outra coisa.
#Para usar as 4 tarefas reservadas, deve-se indicar -n 4 (ou -n ${SLURM_NTASKS}).

srun -n 4 python hello_mpi.py #ou ---> srun -n ${SLURM_NTASKS} python hello_mpi.py



Na próxima célula iniciamos o job cujo ficheiro é job_hello.sh, e cujo nome do job é #SBATCH --job-name=hello4 conforme indicado no script.

>O "programa/código" chamado é o que está no ficheiro hello_mpi.py criado no ponto 1.4 acima.

>Realmente é "chamado" 4 vezes. Cada processo corre uma vez esse código. É o que indica o número 4 em **srun** no job script.

In [11]:
!sbatch job_hello.sh

Submitted batch job 585806


Na próxima célula vemos o estado do job que foi submetido pelo scheduler SLURM aos nós de computação ARM.

In [18]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


**Estados do squeue --me (resumo)**

Os estados mais comuns que se podem ver na fila:

>PD (PENDING) — o job está à espera de recursos/permits (conta, partição, QoS, nós).

>R (RUNNING) — o job está a executar.

>CG (COMPLETING) — a terminar (a libertar recursos/fechar ficheiros).

>CD (COMPLETED) — terminou com sucesso (pode não aparecer dependendo da configuração).

>F (FAILED) — terminou com erro.

>TO (TIMEOUT) — excedeu o tempo limite (walltime).

>NF/CA (NODE_FAIL/CANCELLED) — falha de nó / cancelado.


Quando terminado podemos clicar no ficheiro de saida que neste caso chamamos **hello.txt** ou então usar numa celula **!cat hello.txt**

## Porque é que as linhas não surgem “por ordem”?

Em MPI, os 4 processos escrevem em paralelo para o stdout. A ordem de chegada das mensagens depende do agendador, do sistema de E/S e da rede.
Logo, aparecerá algo como:

Hello ... rank 2 ...
Hello ... rank 0 ...
Hello ... rank 3 ...
Hello ... rank 1 ...


Isto é normal: não há ordenação implícita do stdout em execução paralela.

Como impor ordem (opcional): fazer o root (rank 0) imprimir por sequência de ranks:


In [53]:
%%writefile hello_mpi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
host = MPI.Get_processor_name()

# Mensagem única, como no exemplo original, incluindo o host
msg = f"Hello world from process {rank} of {size} on {host}"

# Impressão ordenada por rank (0,1,2,...) para evitar mistura no stdout
for r in range(size):
    comm.Barrier()         # todos sincronizam neste passo
    if rank == r:
        print(msg, flush=True)

# Sincroniza no final para garantir que todos terminaram antes de o job sair
comm.Barrier()

Overwriting hello_mpi.py


In [54]:
!sbatch job_hello.sh

Submitted batch job 585814


In [60]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


Para ver o resultado clicar duas vezes no ficheiro na estrutra de pastas à esquerda.

Se o resultado não for o esperado acrescentar **?reset** ao URL do browser e voltar a abrir o ficheiro.

### Mas o que faz este novo programa?

Cada processo (rank) constrói a sua mensagem (msg) com “Hello world from process <rank> of <size> on <host>”.

Começa um ciclo for r in range(size). Esse r é o “turno de impressão”: primeiro r=0, depois r=1, … até r=size-1.

Em cada iteração do ciclo:

Todos os processos executam comm.Barrier().
 - Isto faz com que todos esperem uns pelos outros antes de decidir quem imprime.
 - Garante que ninguém “se adianta” no turno seguinte.

Depois da barreira, apenas o processo cujo rank == r imprime a sua mensagem.
 - Os restantes não imprimem nada nessa volta; ficam à espera de chegar à próxima barreira.

Como consequência, na iteração r=0 só o rank 0 imprime. Na iteração r=1 só o rank 1 imprime. E assim sucessivamente, até r=size-1.
 - Isto impõe uma ordem determinística nas linhas do stdout: 0, 1, 2, …
 - O flush=True força a saída a ir imediatamente para o stdout, evitando que o buffer atrase a escrita e baralhe a ordem.

No final (muitas vezes coloca-se ainda uma última Barrier()), todos sincronizam outra vez antes de terminar.
 - Evita que alguns processos saiam do programa enquanto outros ainda estão a imprimir.

### Visualização passo a passo (exemplo com 4 processos: ranks 0,1,2,3)

>Turno r=0

Todos entram na barreira (0,1,2,3).

Saem da barreira ao mesmo tempo.

Só o rank 0 cumpre a condição rank == r → imprime a sua linha.

Ranks 1,2,3 não imprimem nada nesta volta.

>Turno r=1

Todos entram novamente na barreira (0,1,2,3).

Saem juntos.

Só o rank 1 imprime.

Os outros não imprimem.

>Turno r=2

Barreira → sair → imprime o rank 2.

>Turno r=3

Barreira → sair → imprime o rank 3.

Resultado final no ficheiro/terminal:
linha do rank 0 → linha do rank 1 → linha do rank 2 → linha do rank 3 (sempre nesta ordem).

**Porque é que isto evita linhas desordenadas?**

Sem estas barreiras e sem a condição “imprime apenas quem tem o turno”, todos os processos tentariam escrever ao mesmo tempo. A ordem dependeria de latência de rede, agendamento e E/S, logo poderia aparecer 2, 0, 3, 1, etc.
Com o esquema “barreira + turno”, há um único emissor por volta, e por isso a ordem é garantida.

**Observações**

 - Este método sacrifica um pouco de desempenho (há espera nas barreiras) para ganhar legibilidade determinística do stdout — excelente para ensino e depuração.

 - A mensagem inclui host para se ver em que nó cada rank correu — confirma a distribuição real no cluster.